In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')


In [2]:
spark = (
    SparkSession
    .builder
    .appName("landing_to_bronze")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "24G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-abf4be25-d9e1-490d-a5a1-836e52689f62;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 971ms :: artifacts dl 15ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from c

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *

In [5]:
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_item/'
silver_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_item/'

In [6]:
df_silver = (
    spark
    .read
    .format("delta")
    .load(silver_path)
)


In [7]:
list_result = (
    df_silver
    .select('timestamp_kafka')
    .agg({"timestamp_kafka": "max"}).collect()
)

In [8]:
list_result

[Row(max(timestamp_kafka)=datetime.datetime(2022, 3, 17, 10, 38, 44, 56000))]

In [6]:
# Reading last silver data
try:
    df_silver = (
        spark
        .read
        .format("delta")
        .load(silver_path)
    )

    list_result = (
        df_silver
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = list_result[0].asDict()['max(timestamp_kafka)']

except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [7]:
print(last_date)

2021-01-01 00:00:00


In [8]:
df_table = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp_kafka")>last_date)
    .withColumn('pk_single', f.concat(f.col('PEDIDO'), f.col('CODIGO_ENTREGA'), f.col('ITEM')))
)

In [9]:
df_table.count()

12749109

In [10]:
# Snapshot
if df_table.where((f.col('__op') == 'r')|(f.col('__op')=='c')).count() > 0:
    df_table_r = df_table.where(f.col('__op') == 'r')
    (
        df_table_r
        .write
        .format("delta")
        .mode("append")
        .option("checkpointLocation", silver_path + "checkpoint")
        .save(silver_path)
    )
    
# Insert and Update
if df_table.where(f.col('__op') != 'r').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    df_table_u = df_table.where(f.col('__op') != 'r')
    df_table_u_unique = df_table_u.orderBy('offset', ascending = False).dropDuplicates(["PEDIDO", "CODIGO_ENTREGA", "ITEM"])
    (
        silver_table.alias('silver')
        .merge(df_table_u_unique.alias('updates'),'silver.pk_single = updates.pk_single')
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
# Delete
if df_table.where(f.col('__op') == 'd').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    silver_table.delete(col('__op') == 'd')
    

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
    df_silver_teste = (
        spark
        .read
        .format("delta")
        .load(silver_path)
    )

In [ ]:
df_pd = df_silver_teste.toPandas()

In [ ]:
df_pd

In [ ]:
df_pd[df_pd["__op"] != "r"]